# Module 3 Assignment: Building an Automatic Code Review Flow

Welcome to Module 3's assignment! In this lab, you use the Automatic Code Review crew from Modules 1 and 2 and, with some modifications, create a CrewAI Flow that embeds it. 

## Background
Naturally, not all code changes warrant a full agentic review; sometimes it's small fixes, like indentation or fixing typos. In order to avoid wasting resources and time, you will create a Flow that analyzes the depth of the changes and deploys the Crew only if needed. You will also add some parallelism into the tasks of the Crew itself to make things more efficient. 

Up until now, you had created an amazing Automatic Code Review Crew. In this lab, you will use the CrewAI CLI to incorporate this Crew as one of the tasks in a Flow. You will also be improving the performance of the Crew by making some tasks run in parallel, improving the execution time. 

## General instructions for grading
- Replace all `None` instances with your own solution.
- You can add new cells to experiment, but these will be omitted by the grader. Only use the provided cells for your solution code.
- Before submitting, make sure all the cells in your lab work correctly.
- **Do not change variable names**: if you modify variable names, the grader won't be able to find your solutions
- **Use the provided configuration**: for grading, please use all provided configurations. Don't change the configuration files or settings. You can experiment after submitting your lab.
- To submit your notebook, save it and then click on the red **Submit Assignment** button at the top right of the page.

**<font color='#5DADEC'>Please make sure to save your work periodically, so you don't lose any progress.</font>**

## Table of Contents

- [1. Getting familiar with the Flow structure](#1)
- [2. Defining the crew](#2)
    - [Exercise 1: Update the tasks.yaml file](#ex1)
    - [Exercise 2: Update the crew.py file](#ex2)
- [3. Defining the Flow](#3)
  - [Exercise 3: Define the state of the Flow](#ex3)
  - [Exercise 4: Build the Read PR File task](#ex4)
  - [Exercise 5: Build the Analyze Changes task](#ex5)
  - [Exercise 6: Define the Simple Review](#ex6)
  - [Exercise 7: Define the Code and Security Review (Crew)](#ex7)
  - [Exercise 8: Define the Make Final Decision](#ex8)
  - [Exercise 9: Define the kickoff function](#ex9)
- [4. Plotting the Flow](#4)
- [5. Running the Flow](#5)
  - [Exercise 10: Kickoff the Flow](#ex10)

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

`!pip install crewai[tools]==1.3.0`

<a id="1"></a>

## 1. Getting familiar with the Flow structure

In this lab, you will build a CrewAI Flow, by completing the different files in the Flow file structure. You should see a folder named `code_review_flow` with the following structure:

<div style="text-align: center;">
<img src='images/folder-structure.png' width=200>
</div>

All of the files you will need to create the flow are already in place, you will need to complete some code in different parts. As with previous labs, you need to change the `None` placeholders in the code delimited between `### START CODE HERE ###` and `### END CODE HERE ###`.

Here is a brief explanation of the role of each of the files and folders:
- `src/`: Contains all the files needed to define the flow
- `src/code_review_flow/crews/`: In here you can define as many crews as needed for your flow. In this case, you only have the `code_review_crew`
    - `src/code_review_flow/crews/code_review_crew/config`: This folder contains the [`agents.yaml`](code_review_flow/src/code_review_flow/crews/code_review_crew/config/agents.yaml) and [`tasks.yaml`](code_review_flow/src/code_review_flow/crews/code_review_crew/config/tasks.yaml) files you are already familiar with. These help you configure the tasks and agents for the crew. 
    - `src/code_review_flow/crews/code_review_crew/guardrails/`: This folder contains the [`guardrails.py`](code_review_flow/src/code_review_flow/crews/code_review_crew/guardrails/guardrails.py) file, with the guardrails defined for the crew. 
    - [`src/code_review_flow/crews/code_review_crew/crew.py`](code_review_flow/src/code_review_flow/crews/code_review_crew/crew.py): This is the file where you create the crew, assigning it the agents, tasks, along with any additional configuration to these elements.
- [`src/code_review_flow/main.py`](code_review_flow/src/code_review_flow/main.py): In this file you will get to define the Flow object, inside which you will define all the tasks it needs to run, and interconnect them with the help of decorators.


<a id="2"></a>

## 2. Defining the crew

You will be using a very similar Crew to the one you defined in the previous assignments. In fact, you won't need to make any changes to the [`agents.yaml`](code_review_flow/src/code_review_flow/crews/code_review_crew/config/agents.yaml) file. You will, however, make some adaptations to the tasks. 

Originally you had three tasks run sequentially:
- Analyze Code Quality
- Review Security
- Review Decision

Your Crew looked something like this:
<div style="text-align: center;">
<img src="./images/M2agents-tasks-diagram.png" width=600>
</div>

In this lab, you will replace the Review Decision task with a Summarize Findings task, and also run the first two tasks in parallel. The new crew should have this structure:

<div style="text-align: center;">
<img src="./images/parallel-agents-tasks-diagram.png" width=600>
</div>


<a id="ex1"></a>

### Exercise 1: Update the tasks.yaml file

Open the [`tasks.yaml`](code_review_flow/src/code_review_flow/crews/code_review_crew/config/tasks.yaml) file and:

1. Fill in the expected output for the `summarize_findings` task. The output should return a JSON with the following fields:
    - confidence: Your confidence score (**integer** between 0 and 100) of the code changes,
    - findings: A summary of the key findings from both analyses (**string**),
    - fix: A **list** of what must be fixed, and possible solutions, and the explanation for this fixes
    - recommendations: Any additional recommendations or observations (**string**).
    
2. The Summarize Findings task needs to start only after the previous two tasks are finished. Set the `context` to guarantee this happens. The context should contain all the tasks that need to finish before the task can start. 

<a id="ex2"></a>

### Exercise 2: Update the crew.py file

Next, open the [`crew.py`](code_review_flow/src/code_review_flow/crews/code_review_crew/crew.py) file and:

1. Add all the missing decorators (`@task`, `@agent`, `@crew`) for tasks, agents and crew.

2. Set the appropriate arguments in the `analyze_code_quality` and `review_security` tasks so that they run in parallel.

3. Complete the pydantic model to define the output structure of the `summarize_findings` task.

<a id="3"></a>

## 3. Defining the Flow
Now that the Crew is all set up and ready to use, you can go ahead and start building the flow. For this lab you want a CrewAI Flow with these components:

<div style="text-align: center;">
<img src='images/flow-diagram.png' width=300>
</div>

Let's do a little breakdown of each of these tasks:
1. The first task in the flow is in charge of reading the PR file with the code differences. This step replaces the before-kickoff hook you defined in the assignment from the previous module.
2. The next step consists of an LLM call to decide whether the changes are small or if a full analysis is needed. This is a router step, because it can select between two different routes.
3. Based on the decision:

    a. If the changes are simple enough, the Simple Review is executed, and an LLM call is used to decide whether the changes are good to be accepted or if other measures are needed.

    b. If the changes are more complex, the workload is delegated to the Crew for a deep analysis of the code and possible security issues.
4. However the PR was analyzed, the next step is to make the final decision. For this, the output of step 3 is passed to an LLM call to make the final decision.
5. After the final decision is made, an answer is returned to the user.

Another cool thing about Flows is that they allow you to define a state, which is the record of its current progress, inputs, intermediate outputs, and decisions made during execution. You can decide what messages, variables, or values you want to save there.


<a id="ex3"></a>

### Exercise 3: Define the state of the Flow
Begin by defining the state for your Flow. You want to save
- The path to the PR file
- The content of the PR file
- A **list** of errors encountered during execution
- The result of the review (a dictionary)
- A bool to store whether the crew was needed or not
- A summary of token usage, providing insights into the language model’s performance during execution. This information is stored in a dictionary
- The final answer (text)

**Open the [main.py](code_review_flow/src/code_review_flow/main.py) file** and complete missing parts in the `ReviewState` class. For each variable you want to store in the state, you need to define its name, type, and initial value.

For grading purposes, use the given file path, but after submission you can experiment with it and use different files. 

Next, you will start building the flow. For that you need to define a Flow subclass, and provide the State you want to track. Inside this subclass you will define all the different tasks that need to be run, using decorators to interconnect them:
- `@start()`: marks entry points for a Flow. See more information in the [docs](https://docs.crewai.com/en/concepts/flows#%40start).
- `@listen()`: marks a method as a listener for the output of another task in the Flow. The method decorated with `@listen()` will be executed when the specified task emits an output. You can either listen for the output of a particular task, or a specific output of a router. See more information in the [docs](https://docs.crewai.com/en/concepts/flows#%40listen).
- `@router()`: allows you to define conditional routing logic based on the output of the method decorated with `@router()`. See more information in the [docs](https://docs.crewai.com/en/concepts/flows#router).
- `@persist()`: automatically persists all flow method states. You can use this decorator when you want the state of the flow and all executions to be saved into a database, so you can access them and persist data across different executions. 

<a id="ex4"></a>

### Exercise 4: Build the Read PR File task

Next, build the entrypoint of the flow, the `read_pr_file` method. In the **[main.py](code_review_flow/src/code_review_flow/main.py) file** look for the `read_pr_file` definition and:
- Add the corresponding decorator to indicate this is the entrypoint
- If the file is read correctly, save the contents in the `pr_content` state
- If there are any errors, add them to the `errors` state, and update the `final_answer` with an error message.

To update the state of the flow, you can access each state variable by doing `self.state.<variable name>`. For example, to access the PR content you would do `self.state.pr_content`. 

<a id="ex5"></a>

### Exercise 5: Build the Analyze Changes task

Next, build the router method of this flow, the `analyze_changes` method. In the **[main.py](code_review_flow/src/code_review_flow/main.py) file** look for the `analyze_changes` definition and:
- Add the corresponding decorator to indicate this is a router. 
- If there was an issue when reading the file, return"ERROR" and update the final answer with the error found.
- If the PR file was read correctly, call an LLM to make the final decision (no crew needed for this)
- Depending on the decision made by the LLM call, set the `crew_needed` state accordingly and return the answers "SIMPLE" or "COMPLEX" as needed.

<a id="ex6"></a>

### Exercise 6: Define the Simple Review

Now it is time to build the path in case of a simple review. In the **[main.py](code_review_flow/src/code_review_flow/main.py) file** look for the `simple_review` method and:
- Add the corresponding decorator. Since this method should only run conditional on the `analyze_changes` routing method, it should listen for the "SIMPLE" output of the router.
- Complete the missing parts in the LLM prompt. You need to specify the format for the answer. It should return a dictionary with:
    - confidence: Your confidence score (integer between 0 and 100) of the code changes,
    - findings: A summary of the key findings from both analyses,
    - recommendations: Any additional recommendations or observations.

    Do not forget to pass the PR contents, which is saved in the `pr_content` state variable.
- Save the answer of the LLM in the `review_result` state variable.

<a id="ex7"></a>

### Exercise 7: Define the Code and Security Review (Crew)

Next, build the path in case of a complex review. In the **[main.py](code_review_flow/src/code_review_flow/main.py) file** look for the `full_crew_review` method and:
- Add the corresponding decorator. Since this method should only run conditional on the `analyze_changes` routing method, it should listen for the "COMPLEX" output of the router.
- Instantiate the `CodeReviewCrew`, and kickoff the crew. Do not forget to pass the PR content in the inputs. 
- Save the answer of the LLM in the `review_result` state variable.
- If there are any errors running the crew, update the `errors` state and update the final answer with the error.

<a id="ex8"></a>

### Exercise 8: Define the Make Final Decision 

To define the method to make the final decision, go to the **[main.py](code_review_flow/src/code_review_flow/main.py) file** look for the `make_final_decision` method and:
- Add the corresponding decorator. This method needs to run when either `simple_review` or `full_crew_review` finish (only one of them should be triggered). For this, you can use the `or_` function, which allows you to listen to multiple methods and trigger the listener method when any of the specified methods emit an output. For more information, you can go to the [docs](https://docs.crewai.com/en/concepts/flows#conditional-logic%3A-or).
- Make an LLM call with a prompt instructing to make the final decision.
- Save the result of the LLM in the `final_answer` state.

The last method is already given to you. It listens for the `"ERROR"` return in the router, or the return from `make_final_decision`, and then prints the final message. 

<a id="ex9"></a>

### Exercise 9: Define the kickoff function 

Now that the Flow subclass is ready, there are two very useful functions to define: `kickoff` and `plot`. 
- `plot` creates an HTML visualization of the Flow graph, showing the connections between all the methods you defined. 
- `kickoff` sets up the kickoff of the Flow.

Go to the **[main.py](code_review_flow/src/code_review_flow/main.py) file**, and look for the `kickoff()` function. 
- Instantiate the `PRCodeReviewFlow`. It is very important to be able to trace your Flows (and Crews) executions. This allows for monitoring, and identifying issues early on. 
    - Allow for tracing in this flow
- Kickoff the Flow.

**If you haven't done so already, please SAVE the changes in all the files you modified before attempting to continue this lab.**

<a id="4"></a>

## 4. Plotting the Flow

Before actually running the Flow, it is nice to visualize the different steps as graph. This will help you verify that all the connections between the different methods are done correctly. You can do this using the `plot()` function defined on the `main.py` file. Just run the cell below: 

In [ ]:
! cd code_review_flow && crewai flow plot

This will create an HTML file called `crewai_flow.html` inside the `code_review_flow` folder. 

**Note:** The first time you run this cell, it can take a little time because it will create a virtual environment with all the needed dependencies. 

<a id="5"></a>

## 5. Running the Flow

<a id="ex10"></a>

### Exercise 10: Kickoff the Flow

Now you are ready to finally try out the Flow! Since you are working with the CrewAI CLI you actually need to run the code in terminal. There are two ways in which you can do this:
1. Run the cell below to call the `kickoff()` function you defined earlier. If you are wondering what that code is doing here is a breakdown:
- `cd code_review_flow` changes the directory to the `code_review_flow` folder
- `crewai run` calls the CrewAI CLI and kicks off the crew

In [ ]:
! cd code_review_flow && crewai run


2. Open a new terminal con copy these lines:
    ```bash
    cd code_review_flow
    crewai run
    ```
    **Note**: To open a terminal in a new tab you need to click on `File > New > Terminal`

    <div style="text-align: center;">
    <img src='../images/open_new_terminal.png' width=550>
    </div>


### Optional exercises

By the end of the run, you should see something like this:

```shell
╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ b30ddc1b-43a0-4e36-b9ea-109b8b8d5145. View here:                             │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/b30ddc1b-43a0-4e3 │
│ 6-b9ea-109b8b8d5145?access_code=TRACE-********** , Access Code:              │
│ TRACE-**********                                                             │
╰──────────────────────────────────────────────────────────────────────────────╯
```

Go ahead and access the URL. If the hyperlink is cut, meaning it is not spanning both lines, just copy and paste the full URL in a new browser tab. There you can explore your run.

Additionally, you saved the state of the flow in a file named `flow_state.json` . From there you can see how many tokens the Crew used, enabling you to calculate the cost of one run. 

Run the cell below to load the json file, extract the number of tokens, and do a calculation on the cost.

In [ ]:
import json
from pprint import pprint

# load the flow state from the JSON file
with open('flow_state.json', 'r') as f:
    flow_state = json.load(f)

# print the tokens_used variable
pprint(flow_state["tokens_used"])

# Pricing for gpt-4o-mini as of Sept 2025
input_cost_per_1M = 0.15
cached_input_cost_per_1M = 0.075
output_cost_per_1M = 0.6

prompt_tokens = flow_state["tokens_used"]["prompt_tokens"]
cached_tokens = flow_state["tokens_used"]["cached_prompt_tokens"]
completion_tokens = flow_state["tokens_used"]["completion_tokens"]

input_cost = (prompt_tokens / 1_000_000) * input_cost_per_1M
cached_cost = (cached_tokens / 1_000_000) * cached_input_cost_per_1M
output_cost = (completion_tokens / 1_000_000) * output_cost_per_1M

total_cost = input_cost + output_cost + cached_cost

print("------------------------------------\nCost Breakdown:")
print(f"Input cost: ${input_cost:.6f}")
print(f"Output cost: ${output_cost:.6f}")
print(f"Cached cost: ${cached_cost:.6f}")
print(f"Total cost: ${total_cost:.6f}")

Congratulations on reaching the end of the assignment! Now you are ready to Submit your work.

After submitting and being satisfied with your grade you can take some time to experiment changing the guardrails, or adding new URLs to the Website Reading Tool, or even upload a different pull request file with code differences. Don't be afraid to shake things up!